<a href="https://colab.research.google.com/github/etoshey/colabC3D/blob/main/ScaleTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Opensim Sclaing

In [1]:
#@title Install MiniConda (~40s)
import sys
import os

# ensure cwd is /content
os.chdir("/content")

!pip install -q condacolab
import condacolab
condacolab.install()

!conda install -c opensim-org opensim

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - opensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.11.17 |       hbcca054_0         151 KB  conda-for

In [1]:
#@title Install opensim with Conda (~1min)

# Install Opensim
# !conda install -c opensim-org opensim -y
import opensim as osim




In [2]:
#@title Importing Misc Libraries
from google.colab import files #to upload user's files
import numpy as np #used in some functions
import time
!pip install xmltodict


In [3]:
#@title Step1 : upload static trc file
import ipywidgets as widgets
from google.colab import files



file_name = ""

upload_btn = widgets.Button(description='Upload')
def upload_btn_eventhandler(obj):

  global file_name
  uploaded = files.upload()
  file_name = list(uploaded.keys())[0]

  print('Upload Completed ==> '+file_name)


upload_btn.on_click(upload_btn_eventhandler)
display(upload_btn)



Button(description='Upload', style=ButtonStyle())

Saving S+T01.trc to S+T01.trc
Upload Completed ==> S+T01.trc


In [4]:
def _red(txt):
  return "\x1b[31m"+txt+"\x1b[0m"


def _green(txt):
  return "\033[92m"+txt+"\x1b[0m"

def _blue(txt):
  return "\033[94m"+txt+"\033[94m"

def _yellow(txt):
  return "\033[93m"+txt+"\033[93m"

In [39]:
#@title Function1
class body_obj:
  def __init__(self,name,Ref_len,scale_factor):
    self.name = name
    self.Ref_len = Ref_len
    self.scale_factor = scale_factor



Opensim_MarkerSet = {'torso':['V.Sacral','Top.Head'],
                     'pelvis' : ['R.ASIS','L.ASIS'],
                     'femur_r' : ['R.ASIS','R.Knee.Lat'],
                     'femur_l' : ['L.ASIS','L.Knee.Lat'],
                     'tibia_r' : ['R.Knee.Lat','R.Ankle.Lat'],
                     'tibia_l' : ['L.Knee.Lat','L.Ankle.Lat'],
                     'calcn_r' : ['R.Heel','R.Toe.Tip'],
                     'calcn_l' : ['L.Heel','L.Toe.Tip']}

HBM_MarkerSet = {'torso':[['LPSIS','RPSIS'],'THEAD'],
                     'pelvis' : ['RASIS','LASIS'],
                     'femur_r' : ['RASIS','RLEK'],
                     'femur_l' : ['LASIS','LLEK'],
                     'tibia_r' : ['RLEK','RLM'],
                     'tibia_l' : ['LLEK','LLM'],
                     'calcn_r' : ['RHEE','RMT2'],
                     'calcn_l' : ['LHEE','LMT2']}





def get_body_frame_pos(file_path,limb):
  xml_data = open(file_path, 'r').read()  # Read data
  xmlDict = xmltodict.parse(xml_data)
  M_list = xmlDict['OpenSimDocument']['Model']['BodySet']['objects']['Body']

  Fame = [item for item in M_list if item['@name']== limb]
  Frame = Fame[0]['Joint']['CustomJoint']['location_in_parent']
  Frame = Frame.split(' ')
  output=[]
  for item in Frame:
    output.append(float(item))
  return output


def get_all_body_frame_pos(file_path):
  xml_data = open(file_path, 'r').read()  # Read data
  xmlDict = xmltodict.parse(xml_data)
  M_list = xmlDict['OpenSimDocument']['Model']['BodySet']['objects']['Body']
  output = {}
  for b in M_list:
    try:
      Frame = b['Joint']['CustomJoint']['location_in_parent']
      Frame = Frame.split(' ')
      output[b['@name']] = [float(Frame[0]) , float(Frame[1]), float(Frame[2])]
    except:
      # failed to read
      output[b['@name']] = None

  return output


def get_limb_len(child_frame_pos,m1,m2):

  try:

    if child_frame_pos and child_frame_pos != [0.0,0.0,0.0]:
      if type(m1) == list :
        t1 = [item for item in list_markerset_name if item.name == m1[0]][0]
        t2 = [item for item in list_markerset_name if item.name == m1[1]][0]
        M_1 = class_static_markers('',(t1.x+t2.x)/2,(t1.y+t2.y)/2,(t1.z+t2.z)/2,'')
      else :
        M_1 = [item for item in list_markerset_name if item.name == m1][0]

      if type(m2) == list :
        t1 = [item for item in list_markerset_name if item.name == m2[0]][0]
        t2 = [item for item in list_markerset_name if item.name == m2[1]][0]
        M_2 = class_static_markers('',(t1.x+t2.x)/2,(t1.y+t2.y)/2,(t1.z+t2.z)/2,'')
      else :
          M_2 = [item for item in list_markerset_name if item.name == m2][0]

      P2 = [child_frame_pos[0]+M_2.x, child_frame_pos[1]+M_2.y, child_frame_pos[2]+M_2.z]
      P1 = [M_1.x, M_1.y, M_1.z]
      len = math.sqrt(math.pow(P2[0]-P1[0],2) + math.pow(P2[1]-P1[1],2) + math.pow(P2[2]-P1[2],2))
    else:
      M_2 = [item for item in list_markerset_name if item.name == m2][0]
      M_1 = [item for item in list_markerset_name if item.name == m1][0]


      len = math.sqrt(math.pow(M_2.x - M_1.x,2) + math.pow(M_2.y - M_1.y,2) + math.pow( M_2.z - M_1.z,2))

    return len
  except:

    return 1

  def get_limb_len2(SplineX,SplineY,m1,m2):
    fit = osim.SimmSpline()
    for inx,x in enumerate(SplineX):
      fit.addPoint(x,SplineY[inx])

    npArray = np.array([0])
    osimVector = osim.Vector.createFromMat(npArray)
    YO_frame = fit.calcValue(osimVector)

    M_2 = [item for item in list_markerset_name if item.name == m2][0]
    M_1 = [item for item in list_markerset_name if item.name == m1][0]

    P2 = [0 + M_2.x, YO_frame+M_2.y, 0 + M_2.z]
    P1 = [M_1.x, M_1.y, M_1.z]
    len = math.sqrt(math.pow(P2[0]-P1[0],2) + math.pow(P2[1]-P1[1],2) + math.pow(P2[2]-P1[2],2))

    return len



def get_marker_dist(m1,m2):
  return math.sqrt(math.pow(m1.x - m2.x,2) + math.pow(m1.y - m2.y,2) + math.pow(m1.z - m2.z,2))


def get_gait2392_limb_len(body_frames,markerSet):


  if markerSet == 'OpenSim':
    selected_markerSet = Opensim_MarkerSet
  elif markerSet == 'HBM':
    selected_markerSet = HBM_MarkerSet


  output = {}
  #torso
  torso_len = get_limb_len(body_frames['torso'],selected_markerSet['torso'][0],selected_markerSet['torso'][1])
  output['torso'] = body_obj('torso',torso_len,1)
  #pelvis
  pelvis_len = get_limb_len(body_frames['pelvis'],selected_markerSet['pelvis'][0],selected_markerSet['pelvis'][1])
  output['pelvis'] = body_obj('pelvis',pelvis_len,1)
  #femur_r
  femur_r_len = get_limb_len(body_frames['femur_r'],selected_markerSet['femur_r'][0],selected_markerSet['femur_r'][1])
  output['femur_r'] = body_obj('femur_r',femur_r_len,1)
  #femur_l
  femur_r_len = get_limb_len(body_frames['femur_l'],selected_markerSet['femur_l'][0],selected_markerSet['femur_l'][1])
  output['femur_l'] = body_obj('femur_l',femur_r_len,1)
  #tibia_r
  tibia_r_len = get_limb_len([-0.004499861510153792,-0.3958206952573354,0.0],selected_markerSet['tibia_r'][0],selected_markerSet['tibia_r'][1])
  output['tibia_r'] = body_obj('tibia_r',tibia_r_len,1)
  #tibia_l
  tibia_l_len = get_limb_len([-0.004499861510153792,-0.3958206952573354,0.0],selected_markerSet['tibia_l'][0],selected_markerSet['tibia_l'][1])
  output['tibia_l'] = body_obj('tibia_l',tibia_l_len,1)
  #foot_r
  calcn_r_len = get_limb_len(body_frames['calcn_r'],selected_markerSet['pelvis'][0],selected_markerSet['calcn_r'][1])
  output['calcn_r'] = body_obj('calcn_r',calcn_r_len,1)
  #foot_l
  calcn_l_len = get_limb_len(body_frames['calcn_l'],selected_markerSet['pelvis'][0],selected_markerSet['calcn_l'][1])
  output['calcn_l'] = body_obj('calcn_l',calcn_l_len,1)

  return output


def compute_scale_factor(limb,m1,m2):
  return (get_marker_dist(m1,m2)/1000) / Frames_pos[limb].Ref_len



def Generate_Scale_XML(mass,height,_model_name,marker_set_file,MarkerPair,trc_markers,trc_file,time1,time2):

  #Load File
  if os.path.isfile("/content/subject01_Setup_Scale.xml") == False:
    !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/Setup/subject01_Setup_Scale.xml

  #Read Xml File
  xml_data = open('subject01_Setup_Scale.xml', 'r').read()  # Read data
  xmlDict = xmltodict.parse(xml_data)

  #update --> mass
  xmlDict['OpenSimDocument']['ScaleTool']['mass'] = mass
  xmlDict['OpenSimDocument']['ScaleTool']['height'] = height
  xmlDict['OpenSimDocument']['ScaleTool']['GenericModelMaker']['model_file'] = _model_name
  xmlDict['OpenSimDocument']['ScaleTool']['GenericModelMaker']['marker_set_file'] = marker_set_file



  Measurement = []
  for b in MarkerPair:
    BodyScale = MarkerPair[b]
    objects = []
    for obj in BodyScale['body']:
      objects.append({'@name':obj,'axes':{'#text':'X Y Z'}})


    Measurement.append({'@name': b ,
                                      'apply' : {'#text':'true'},
                                      'MarkerPairSet': {'@name':"",'objects':{'MarkerPair':{'@name':"",'markers':{'#text': MarkerPair[b]['m1']+' '+MarkerPair[b]['m2']}} }},
                                      'BodyScaleSet':{'@name':"",
                                                      'objects': { 'BodyScale' : objects }
                                                    }
                                      })




  xmlDict['OpenSimDocument']['ScaleTool']['ModelScaler']['MeasurementSet']['objects']['Measurement'] = Measurement

  xmlDict['OpenSimDocument']['ScaleTool']['ModelScaler']['marker_file'] = trc_file
  xmlDict['OpenSimDocument']['ScaleTool']['ModelScaler']['time_range'] = str(time1) +' '+str(time2)

  xmlDict['OpenSimDocument']['ScaleTool']['MarkerPlacer']['marker_file'] = trc_file
  xmlDict['OpenSimDocument']['ScaleTool']['MarkerPlacer']['time_range'] = str(time1) +' '+str(time2)
  xmlDict['OpenSimDocument']['ScaleTool']['MarkerPlacer']['output_motion_file'] = 'static_output.mot'
  xmlDict['OpenSimDocument']['ScaleTool']['MarkerPlacer']['output_model_file'] = 'simbody.osim'

  #Remove Manual
  xmlDict['OpenSimDocument']['ScaleTool']['ModelScaler']['ScaleSet']= ''

  #Marker Weights
  Markers = []
  for m in trc_markers:
    Markers.append({'@name':m, 'apply' : 'true' , 'weight' : 1.00000 })

  xmlDict['OpenSimDocument']['ScaleTool']['MarkerPlacer']['IKTaskSet']['objects']['IKMarkerTask']= Markers


  #Write XML
  if os.path.isfile("/content/subject01_Setup_Scale.xml") == True:
    os.remove('/content/subject01_Setup_Scale.xml')
  writer = open('New_Scale_Setup.xml', 'w').write(xmltodict.unparse(xmlDict, pretty=True))




def Generate_MarkerSet(markerSet_name,_list):
  #Read Xml File
  xml_data = open(markerSet_name+'.xml', 'r').read()  # Read data
  xmlDict = xmltodict.parse(xml_data)
  Mlist = xmlDict['OpenSimDocument']['MarkerSet']['objects']['Marker']

  print(_list)


  for i in range(len(_list)):
    for j in range(len(_list[i])):
      m = [x for x in Mlist if x['@name'] == _list[i][j].description][0]
      print(m)

      xmlDict['OpenSimDocument']['MarkerSet']['objects']['Marker'][int(Mlist.index(m))]['@name'] = _list[i][j].value


  #Write XML
  if os.path.isfile("/content/MarkerSet.xml") == True:
    os.remove('/content/MarkerSet.xml')
  writer = open('MarkerSet.xml', 'w').write(xmltodict.unparse(xmlDict, pretty=True))



def find_index(_list,p):
  try:
    return _list.index(p)
  except ValueError:
    #Similar name
    #split name
    return 0


In [27]:
#@title Function2
def update_scale_factor_torso(change):
  m1 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[0][0].value][0]
  m2 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[0][1].value][0]
  scale = compute_scale_factor('torso',m1,m2)
  _list_scale_dropdown[0][2].value = str(scale)
  Scale_MarkerPair['torso'] = {'val' : scale, 'm1' :  _list_scale_dropdown[0][0].value , 'm2': _list_scale_dropdown[0][1].value , 'body':['torso']}


def update_scale_factor_pelvis(change):
  m1 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[1][0].value][0]
  m2 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[1][1].value][0]
  scale = compute_scale_factor('pelvis',m1,m2)
  _list_scale_dropdown[1][2].value = str(scale)
  Scale_MarkerPair['pelvis'] = {'val' : scale, 'm1' :  _list_scale_dropdown[1][0].value , 'm2': _list_scale_dropdown[1][1].value, 'body':['pelvis']}

def update_scale_factor_Lfemur(change):
  m1 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[2][0].value][0]
  m2 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[2][1].value][0]
  scale = compute_scale_factor('femur_l',m1,m2)
  _list_scale_dropdown[2][2].value = str(scale)
  Scale_MarkerPair['thigh_l'] = {'val' : scale, 'm1' :  _list_scale_dropdown[2][0].value , 'm2': _list_scale_dropdown[2][1].value, 'body':['femur_l']}



def update_scale_factor_Rfemur(change):
  m1 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[3][0].value][0]
  m2 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[3][1].value][0]
  scale = compute_scale_factor('femur_r',m1,m2)
  _list_scale_dropdown[3][2].value = str(scale)
  Scale_MarkerPair['thigh_r'] = {'val' : scale, 'm1' :  _list_scale_dropdown[3][0].value , 'm2': _list_scale_dropdown[3][1].value, 'body':['femur_r']}


def update_scale_factor_Ltibia(change):
  m1 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[4][0].value][0]
  m2 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[4][1].value][0]
  scale = compute_scale_factor('tibia_l',m1,m2)
  _list_scale_dropdown[4][2].value = str(scale)
  Scale_MarkerPair['shank_l'] = {'val' : scale, 'm1' :  _list_scale_dropdown[4][0].value , 'm2': _list_scale_dropdown[4][1].value, 'body':['tibia_l','talus_l']}



def update_scale_factor_Rtibia(change):
  m1 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[5][0].value][0]
  m2 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[5][1].value][0]
  scale = compute_scale_factor('tibia_r',m1,m2)
  _list_scale_dropdown[5][2].value = str(scale)
  Scale_MarkerPair['shank_r'] = {'val' : scale, 'm1' :  _list_scale_dropdown[5][0].value , 'm2': _list_scale_dropdown[5][1].value, 'body':['tibia_r','talus_r']}


def update_scale_factor_Lfoot(change):
  m1 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[6][0].value][0]
  m2 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[6][1].value][0]
  scale = compute_scale_factor('calcn_r',m1,m2)
  _list_scale_dropdown[6][2].value = str(scale)
  Scale_MarkerPair['foot_r'] = {'val' : scale, 'm1' :  _list_scale_dropdown[6][0].value , 'm2': _list_scale_dropdown[6][1].value, 'body':['calcn_r','toes_r']}


def update_scale_factor_Rfoot(change):
  m1 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[7][0].value][0]
  m2 = [x for x in static_markers_arr if x.name == _list_scale_dropdown[7][1].value][0]
  scale = compute_scale_factor('calcn_l',m1,m2)
  _list_scale_dropdown[7][2].value = str(scale)
  Scale_MarkerPair['foot_l'] = {'val' : scale, 'm1' :  _list_scale_dropdown[7][0].value , 'm2': _list_scale_dropdown[7][1].value, 'body':['calcn_l','toes_l']}



In [40]:
#@title OpenSim SclaeTool Form
import os
import ipywidgets as widgets
import xmltodict
import math


class class_static_markers:
  def __init__(self,name,x,y,z,body=''):
    self.name = name
    self.x = float(x)
    self.y = float(y)
    self.z = float(z)
    self.body = body

class class_scale_limb:
  def __init__(self,name,R1,R2,M1,M2):
    self.name = name
    self.R1 = R1
    self.R2 = R2
    self.M1 = M1
    self.M2 = M2


list_scale_limb = []
static_markers_arr = []
static_markers = []
list_markerset_name = []
Frames_pos = {}
_list = [[],[],[],[]]
_list_scale_dropdown = []
Scale_MarkerPair = {}
Scale_data = {}
model_body = []
NumFrames=0

model_name=''
#get all files
all_xml_files =[]
all_osim_files =[]
all_trc_files =['select..']
for file_path in os.listdir('/content/'):
    if os.path.isfile(os.path.join('/content/', file_path)):
      if os.path.splitext(file_path)[1]=='.xml':
        all_xml_files.append(file_path)
      elif os.path.splitext(file_path)[1]=='.osim':
        all_osim_files.append(file_path)
      elif os.path.splitext(file_path)[1]=='.trc':
        all_trc_files.append(file_path)



#--------------------------------------------------------------------------------------------------------#
all_trc_dropdown = widgets.Dropdown(description='Selecte static data(trc):',options = all_trc_files)
select_trc_button = widgets.Button(
    description='OK',
    button_style='info'
)

def dropdown_handler(change):
    global static_trc_path
    static_trc_path = change.new
all_trc_dropdown.observe(dropdown_handler, names='value')


def select_trc_btn_eventhandler(obj):
  global static_markers_arr,static_markers,static_trc_path,NumFrames
  my_file = open(static_trc_path, "r")
  my_file.readline();#line1
  my_file.readline();#line2
  info = my_file.readline();#line3
  info = info.split('\t')
  info = [item for item in info if item != '' and item != '\n']
  NumFrames = info[2]

  param = my_file.readline();
  my_file.readline();
  my_file.readline();
  param2 = my_file.readline();

  static_markers = param.split('\t')
  static_markers = [item for item in static_markers if item != '' and item != '\n']
  static_markers = static_markers[2:]

  marker_pos = param2.split('\t');
  marker_pos = [item for item in marker_pos if item != '' and item != '\n']
  marker_pos = marker_pos[2:]


  for inx,m in enumerate(static_markers):
    static_markers_arr.append(class_static_markers(m,marker_pos[(inx*3)] , marker_pos[(inx*3)+1] , marker_pos[(inx*3)+2]))

  show_form()


select_trc_button.on_click(select_trc_btn_eventhandler)

#--------------------------------------------------------------------------------------------------------#

h0 = widgets.HBox([all_trc_dropdown, select_trc_button ])
display(h0)

#--------------------------------------------------------------------------------------------------------#


def show_form():
  #-------> Section 1
  section1_lbl =  widgets.Label(value="----------------------------- Anthropometry & Model -----------------------------");
  mass = widgets.Text(value='76',placeholder='Type something',description='Weight(Kg):',disabled=False)
  Scale_data['mass'] = mass
  height = widgets.Text(value='1800',placeholder='Type something',description='height(Cm):',disabled=False)
  Scale_data['height'] = height

  #-------> GenericModelMaker
  model_dropdown = widgets.Dropdown(description='Model:',options=['','gait2392_simbody', 'gait2354_simbody','gait10dof18musc'])
  model_dropdown.observe(dropdown_Selectmodel, names='value')
    #Generate markerSet.xml based on static trc



  spliter_lbl =  widgets.Label(value="-------------------------------------------------------------------------------------------------------------------");
  #marker_file
  marker_file = widgets.Label(value="static trial : " + static_trc_path)
  #MarkerSet
  MarkerSet_dropdown = widgets.Dropdown(description='MarkerSet :',options = ['HBM','Plug-in Gait' , 'Cast' , 'Cleveland'])
  MarkerSet_dropdown.observe(dropdown_MarkerSet, names='value')



  h1 = widgets.HBox([mass, height , model_dropdown])
  h2 = widgets.VBox([marker_file,MarkerSet_dropdown])

  display(section1_lbl,h1,spliter_lbl,h2)




def Setup_Xml_Generator():
  print("ss")



def show_model_scaler():
    #-------> ModelScaler
  ModelScaler_lbl =  widgets.Label(value="----------------------------- ModelScaler -----------------------------");
  # torso
  torso1_marker_dropdown = widgets.Dropdown(description='Torso 1:',options = static_markers)
  torso1_marker_dropdown.observe(update_scale_factor_torso, names='value')
  torso2_marker_dropdown = widgets.Dropdown(description='Torso 2:',options = static_markers)
  torso2_marker_dropdown.observe(update_scale_factor_torso, names='value')
  torso3_marker_dropdown = widgets.Dropdown(description='Torso 3:',options = static_markers)
  torso_scale = widgets.Text(value=str(Frames_pos['torso'].scale_factor),placeholder='Type something',description='Sclae Factor:')

  #pelvis
  pelvis1_marker_dropdown = widgets.Dropdown(description='Pelvis 1:',options = static_markers)
  pelvis1_marker_dropdown.observe(update_scale_factor_pelvis, names='value')
  pelvis2_marker_dropdown = widgets.Dropdown(description='Pelvis 2:',options = static_markers)
  pelvis2_marker_dropdown.observe(update_scale_factor_pelvis, names='value')
  pelvis_scale = widgets.Text(value=str(Frames_pos['pelvis'].scale_factor),placeholder='Type something',description='Sclae Factor:')
  #thigh
  Lthigh1_marker_dropdown = widgets.Dropdown(description='Left thigh1:',options = static_markers)
  Lthigh1_marker_dropdown.observe(update_scale_factor_Lfemur, names='value')
  Lthigh2_marker_dropdown = widgets.Dropdown(description='Left thigh2:',options = static_markers)
  Lthigh2_marker_dropdown.observe(update_scale_factor_Lfemur, names='value')
  Lthigh_scale = widgets.Text(value=str(Frames_pos['femur_l'].scale_factor),placeholder='Type something',description='Sclae Factor:')

  Rthigh1_marker_dropdown = widgets.Dropdown(description='Right thigh1:',options = static_markers)
  Rthigh1_marker_dropdown.observe(update_scale_factor_Rfemur, names='value')
  Rthigh2_marker_dropdown = widgets.Dropdown(description='Right thigh2:',options = static_markers)
  Rthigh2_marker_dropdown.observe(update_scale_factor_Rfemur, names='value')
  Rthigh_scale = widgets.Text(value=str(Frames_pos['femur_r'].scale_factor),placeholder='Type something',description='Sclae Factor:')
  #shank
  Lshank1_marker_dropdown = widgets.Dropdown(description='Left shank1:',options = static_markers)
  Lshank1_marker_dropdown.observe(update_scale_factor_Ltibia, names='value')
  Lshank2_marker_dropdown = widgets.Dropdown(description='Left shank2:',options = static_markers)
  Lshank2_marker_dropdown.observe(update_scale_factor_Ltibia, names='value')
  Lshank_scale = widgets.Text(value=str(Frames_pos['tibia_l'].scale_factor),placeholder='Type something',description='Sclae Factor:')

  Rshank1_marker_dropdown = widgets.Dropdown(description='Right shank1:',options = static_markers)
  Rshank1_marker_dropdown.observe(update_scale_factor_Rtibia, names='value')
  Rshank2_marker_dropdown = widgets.Dropdown(description='Right shank2:',options = static_markers)
  Rshank2_marker_dropdown.observe(update_scale_factor_Rtibia, names='value')
  Rshank_scale = widgets.Text(value=str(Frames_pos['tibia_r'].scale_factor),placeholder='Type something',description='Sclae Factor:')
  #foot
  Lfoot1_marker_dropdown = widgets.Dropdown(description='Left foot1:',options = static_markers)
  Lfoot1_marker_dropdown.observe(update_scale_factor_Lfoot, names='value')
  Lfoot2_marker_dropdown = widgets.Dropdown(description='Left foot2:',options = static_markers)
  Lfoot2_marker_dropdown.observe(update_scale_factor_Lfoot, names='value')
  Lfoot_scale = widgets.Text(value=str(Frames_pos['calcn_l'].scale_factor),placeholder='Type something',description='Sclae Factor:')

  Rfoot1_marker_dropdown = widgets.Dropdown(description='Right foot1:',options = static_markers)
  Rfoot1_marker_dropdown.observe(update_scale_factor_Rfoot, names='value')
  Rfoot2_marker_dropdown = widgets.Dropdown(description='Right foot2:',options = static_markers)
  Rfoot2_marker_dropdown.observe(update_scale_factor_Rfoot, names='value')
  Rfoot_scale = widgets.Text(value=str(Frames_pos['calcn_l'].scale_factor),placeholder='Type something',description='Sclae Factor:')

  h3 = widgets.HBox([torso1_marker_dropdown,torso2_marker_dropdown,torso_scale])
  _list_scale_dropdown.append([torso1_marker_dropdown,torso2_marker_dropdown,torso_scale])
  h4 = widgets.HBox([pelvis1_marker_dropdown,pelvis2_marker_dropdown,pelvis_scale])
  _list_scale_dropdown.append([pelvis1_marker_dropdown,pelvis2_marker_dropdown,pelvis_scale])
  h5 = widgets.HBox([Lthigh1_marker_dropdown,Lthigh2_marker_dropdown,Lthigh_scale])
  _list_scale_dropdown.append([Lthigh1_marker_dropdown,Lthigh2_marker_dropdown,Lthigh_scale])
  h6 = widgets.HBox([Rthigh1_marker_dropdown,Rthigh2_marker_dropdown,Rthigh_scale])
  _list_scale_dropdown.append([Rthigh1_marker_dropdown,Rthigh2_marker_dropdown,Rthigh_scale])
  h7 = widgets.HBox([Lshank1_marker_dropdown,Lshank2_marker_dropdown,Lshank_scale])
  _list_scale_dropdown.append([Lshank1_marker_dropdown,Lshank2_marker_dropdown,Lshank_scale])
  h8 = widgets.HBox([Rshank1_marker_dropdown,Rshank2_marker_dropdown,Rshank_scale])
  _list_scale_dropdown.append([Rshank1_marker_dropdown,Rshank2_marker_dropdown,Rshank_scale])
  h9 = widgets.HBox([Lfoot1_marker_dropdown,Lfoot2_marker_dropdown,Lfoot_scale])
  _list_scale_dropdown.append([Lfoot1_marker_dropdown,Lfoot2_marker_dropdown,Lfoot_scale])
  h10 = widgets.HBox([Rfoot1_marker_dropdown,Rfoot2_marker_dropdown,Rfoot_scale])
  _list_scale_dropdown.append([Rfoot1_marker_dropdown,Rfoot2_marker_dropdown,Rfoot_scale])

  # Time Range
  time_slider = widgets.FloatRangeSlider(
      value=[0, NumFrames],
      min=0,
      max=NumFrames,
      step=1,
      description='َFrame : ',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      layout=widgets.Layout(width='50%')
    )

  Scale_data['time'] = time_slider
  display(ModelScaler_lbl,h3,h4,h5,h6,h7,h8,h9,h10,time_slider)

  show_proc_btn()



def Compute_Scale_Factor(model_p1,model_p2,data_p1,data_p2):
  model_dis_x = model_p1[0] - model_p2[0]
  model_dis_y = model_p1[1] - model_p2[1]
  model_dis_z = model_p1[2] - model_p2[2]
  model_len = math.sqrt(model_dis_x*model_dis_x + model_dis_y * model_dis_y + model_dis_z * model_dis_z)

  data_dis_x = data_p1[0] - data_p2[0]
  data_dis_y = data_p1[1] - data_p2[1]
  data_dis_z = data_p1[2] - data_p2[2]
  data_len = math.sqrt(data_dis_x * data_dis_x + data_dis_y * data_dis_y + data_dis_z * data_dis_z)


def show_scaler_section(obj):

  global Frames_pos
  #read model .osim
  print(model_name)
  if model_name =='gait2392_simbody':
    if os.path.isfile("/content/gait2392_simbody.osim") == False:
      !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/Model/gait2392_simbody.osim
    Frames_pos = get_gait2392_limb_len(get_all_body_frame_pos('gait2392_simbody.osim'),MarkerSet_Name)
  elif model_name =='gait2354_simbody':
    if os.path.isfile("/content/gait2354_simbody.osim") == False:
      !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/Model/gait2354_simbody.osim


  xml_data = open(model_name+'.osim', 'r').read()  # Read data
  xmlDict = xmltodict.parse(xml_data)
  M_list = xmlDict['OpenSimDocument']['Model']['BodySet']['objects']['Body']
  for item in M_list:
    model_body.append(item['@name'])

    # _R = [x for x in list_markerset_name if x.body == item['@name'] ]
    # if len(_R)>0:
    #   print(item['@name'])
    #   for i in range(len(_R)):
    #     a=0

    #list_scale_limb.append(class_scale_limb(item['@name'],_R[0].))


  print(_blue("[Generateing a MarkerSet File]"))
  Generate_MarkerSet('gait2392_Scale_MarkerSet',_list)


  show_model_scaler()


def dropdown_Selectmodel(change):
  global model_name
  model_name = change.new


def dropdown_MarkerSet(change):
  global static_trc_path,list_markerset_name,MarkerSet_Name

  xml_file_name = ""
  match change.new:
    case "HBM":
      if os.path.isfile("/content/HBM.xml") == False:
        # https://github.com/etoshey/colabC3D/blob/7bb1b142b365f9f48f40004e836e4919419aa86f/Files/Opensim/MarkerSet/markersHBM.xml
        !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/MarkerSet/HBM.xml
      xml_file_name = "HBM.xml"
      MarkerSet_Name = "HBM"
    case "Cleveland":
      if os.path.isfile("/content/gait2392_Scale_MarkerSet.xml") == False:
        !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/MarkerSet/gait2392_Scale_MarkerSet.xml
      xml_file_name = "gait2392_Scale_MarkerSet.xml"
      MarkerSet_Name = "OpenSim"

  print(xml_file_name)
  if xml_file_name != "":
    #read xml file
    xml_data = open(xml_file_name, 'r').read()  # Read data
    xmlDict = xmltodict.parse(xml_data)
    M_list = xmlDict['OpenSimDocument']['MarkerSet']['objects']['Marker']


    if (len(M_list) <= len(static_markers)): # number of marker
      for i in range(0, len(M_list),4):
        if (len(M_list) - i >= 4):
          for j in range(i,i+4):
            _list[j-i].append(widgets.Dropdown(description=M_list[j]['@name'],options = static_markers , value=static_markers[find_index(static_markers,M_list[j]['@name'])]))
            pos = M_list[j]['location'].split(' ')
            pos = [item for item in pos if item != '' and item != '\n']

            list_markerset_name.append(class_static_markers(M_list[j]['@name'],pos[0],pos[1],pos[2],M_list[j]['body']))
        else:
          for j in range(i,len(M_list)):
            _list[j-i].append(widgets.Dropdown(description=M_list[j]['@name'],options = static_markers , value=static_markers[find_index(static_markers,M_list[j]['@name'])]))
            pos = M_list[j]['location'].split(' ')
            pos = [item for item in pos if item != '' and item != '\n']
            list_markerset_name.append(class_static_markers(M_list[j]['@name'],pos[0],pos[1],pos[2],M_list[j]['body']))



      # make table
      V1 = widgets.VBox(_list[0])
      V2 = widgets.VBox(_list[1])
      V3 = widgets.VBox(_list[2])
      V4 = widgets.VBox(_list[3])

      H = widgets.HBox([V1,V2,V3,V4])
      Title =  widgets.Label(value="Label Matching ==>");
      #Next
      Next_button = widgets.Button(description='Next Step',button_style='success')
      Next_button.on_click(show_scaler_section)
      display(Title,H,Next_button)
    else:
      print(_red("[Failed]") + "Number of markers must be equal in static trial and MarkerSet.xml "+ str(len(M_list)) + " - " + str(len(static_markers)))




def show_proc_btn():
  #Processing
  run_button = widgets.Button(description='Run!',button_style='success')
  run_button.on_click(Make_Scale_XML)
  display(run_button)

def Make_Scale_XML(obj):
  Generate_Scale_XML(Scale_data['mass'].value,Scale_data['height'].value,model_name+'.osim','MarkerSet.xml',Scale_MarkerPair,static_markers,static_trc_path,t1,t2)



  # ####################################################################################


Label(value='----------------------------- Anthropometry & Model -----------------------------')

Label(value='-------------------------------------------------------------------------------------------------…


HBM.xml


Label(value='Label Matching ==>')

Button(button_style='success', description='Next Step', style=ButtonStyle())

gait2392_simbody
[Generateing a MarkerSet File]
[[Dropdown(description='XIPH', index=23, options=('LASIS', 'RASIS', 'LPSIS', 'RPSIS', 'LLTHI', 'LLEK', 'LMEK', 'LLSHA', 'LLM', 'LMM', 'LHEE', 'LMT2', 'LMT5', 'RLTHI', 'RLEK', 'RMEK', 'RLSHA', 'RLM', 'RMM', 'RHEE', 'RMT2', 'RMT5', 'C7', 'XIPH', 'JN', 'LHEAD', 'THEAD', 'RHEAD', 'FHEAD', 'LSHO', 'LDELT', 'LLEE', 'LMEE', 'LFRM', 'LLW', 'LMW', 'LFIN', 'RSHO', 'RDELT', 'RLEE', 'RFRM', 'RLW', 'RMW', 'RFIN', 'gamepad1', 'gamepad2', 'gamepad3', 'platform1', 'platform2', 'platform3', 'T10', 'RMEE'), value='XIPH'), Dropdown(description='RASIS', index=1, options=('LASIS', 'RASIS', 'LPSIS', 'RPSIS', 'LLTHI', 'LLEK', 'LMEK', 'LLSHA', 'LLM', 'LMM', 'LHEE', 'LMT2', 'LMT5', 'RLTHI', 'RLEK', 'RMEK', 'RLSHA', 'RLM', 'RMM', 'RHEE', 'RMT2', 'RMT5', 'C7', 'XIPH', 'JN', 'LHEAD', 'THEAD', 'RHEAD', 'FHEAD', 'LSHO', 'LDELT', 'LLEE', 'LMEE', 'LFRM', 'LLW', 'LMW', 'LFIN', 'RSHO', 'RDELT', 'RLEE', 'RFRM', 'RLW', 'RMW', 'RFIN', 'gamepad1', 'gamepad2', 'gamepad3', 'pla

IndexError: ignored

In [25]:
A = {'x':1 , 'y':2}

print(A.x)

AttributeError: ignored

In [ ]:
import datetime
from dateutil import parser

current_time = datetime.datetime.now().timestamp()

Scaler = osim.ScaleTool('/content/New_Scale_Setup.xml')
status = Scaler.run()

print(status)

mylog = open('/var/colab/app.log', 'r')
Lines = mylog.readlines()

#print log
for log in Lines:
  row = json.loads(log)
  log_time = parser.isoparse(row["time"]).timestamp()
  # log_time = datetime.datetime.strptime(row["time"], '"%Y-%m-%dT%H:%M:%SZ"')
  if log_time > current_time:
    if "[info]" in row["msg"]:
      print(_green(row["msg"]))
    elif "[warning]" in row["msg"]:
      print(_yellow(row["msg"]))
    elif "[error]" in row["msg"]:
      print(_red(row["msg"]))
